In [1]:
import os
os.chdir('../..')


In [2]:
from classes.helpers import *
import numpy as np
import json
import pandas as pd
import scipy.stats as ss
from sklearn import linear_model
import ast
import scipy.stats as ss

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

data_folder = '../data/'
min_nbr_rats = 5

# Load the DF with the ranks

In [3]:
df = pd.read_csv(data_folder + 'tmp/time_series_{}_ranks.csv'.format(min_nbr_rats), header=[0,1])

# Transform strings into arrays
for i in df.index:
    for key1 in ['ba', 'rb']:
        for key2 in ['dates', 'ratings', 'z_scores', 'ranks']:
            arr = ast.literal_eval(df.loc[i][key1][key2])
            df.set_value(i, (key1, key2), arr)
            
diffs = {'ba': [], 'rb': []}

for key in diffs.keys():
    for i in df.index:
        row = df.iloc[i]
        diffs[key].append(row[key]['z_scores'][0])

thresholds = {}

for key in ['ba', 'rb']:
    thresholds[key] = {}
    thresholds[key]['low'] = np.percentile(diffs[key], 15)
    thresholds[key]['high'] = np.percentile(diffs[key], 85)

In [7]:
min_nbr_rats = 10

for to_study in ['H-L', 'H-M', 'M-L']:

    nbr_draws = 1000

    nbrs = {'ba': {}, 'rb': {}}

    for cl in [to_study, to_study[::-1]]:

        cls = {'ba': cl[0], 'rb': cl[-1]}

        subdf = df[((df['ba']['class'] == cl[0]) & (df['rb']['class'] == cl[-1])) & 
                   (df['ba']['nbr_ratings'] >= min_nbr_rats) & (df['rb']['nbr_ratings'] >= min_nbr_rats)] 

        for key in ['ba', 'rb']:

            nbrs[key][cls[key]] = {}

            ranks = np.array([r[min_nbr_rats-1] for r in subdf[key]['ranks']])

            nbrs[key][cls[key]]['avg'] = np.mean(ranks)

            tmp = []
            # Go through each draw
            for d in range(nbr_draws):

                # Get the indices
                idx = np.random.randint(0, len(subdf[key]), len(subdf[key]))

                ranks_tmp = ranks[idx]

                tmp.append(np.mean(ranks_tmp))
            nbrs[key][cls[key]]['low'] = np.percentile(tmp, 2.5)
            nbrs[key][cls[key]]['high'] = np.percentile(tmp, 97.5)
        

    print('{}:'.format(to_study))
    for key in nbrs:
        print('  {}:'.format(key.upper()))
        for key2 in nbrs[key]:
            print('    {}: {:.3f} [{:.3f}, {:.3f}]'.format(key2, nbrs[key][key2]['avg'], nbrs[key][key2]['low'], nbrs[key][key2]['high']))
    print()

H-L:
  BA:
    H: 0.545 [0.491, 0.599]
    L: 0.450 [0.379, 0.523]
  RB:
    L: 0.489 [0.419, 0.554]
    H: 0.522 [0.450, 0.596]

H-M:
  BA:
    H: 0.643 [0.624, 0.662]
    M: 0.578 [0.559, 0.596]
  RB:
    M: 0.595 [0.575, 0.614]
    H: 0.627 [0.608, 0.646]

M-L:
  BA:
    M: 0.396 [0.376, 0.416]
    L: 0.368 [0.348, 0.391]
  RB:
    L: 0.350 [0.330, 0.370]
    M: 0.400 [0.380, 0.419]



In [6]:
min_nbr_rats = 5

for to_study in ['H-L', 'H-M', 'M-L']:

    nbr_draws = 1000

    nbrs = {'ba': {}, 'rb': {}}

    for cl in [to_study, to_study[::-1]]:

        cls = {'ba': cl[0], 'rb': cl[-1]}

        subdf = df[((df['ba']['class'] == cl[0]) & (df['rb']['class'] == cl[-1])) & 
                   (df['ba']['nbr_ratings'] >= min_nbr_rats) & (df['rb']['nbr_ratings'] >= min_nbr_rats)] 

        for key in ['ba', 'rb']:

            nbrs[key][cls[key]] = {}

            ranks = np.array([r[min_nbr_rats-1] for r in subdf[key]['ranks']])

            nbrs[key][cls[key]]['avg'] = np.mean(ranks)

            tmp = []
            # Go through each draw
            for d in range(nbr_draws):

                # Get the indices
                idx = np.random.randint(0, len(subdf[key]), len(subdf[key]))

                ranks_tmp = ranks[idx]

                tmp.append(np.mean(ranks_tmp))
            nbrs[key][cls[key]]['low'] = np.percentile(tmp, 2.5)
            nbrs[key][cls[key]]['high'] = np.percentile(tmp, 97.5)
        

    print('{}:'.format(to_study))
    for key in nbrs:
        print('  {}:'.format(key.upper()))
        for key2 in nbrs[key]:
            print('    {}: {:.3f} [{:.3f}, {:.3f}]'.format(key2, nbrs[key][key2]['avg'], nbrs[key][key2]['low'], nbrs[key][key2]['high']))
    print()

H-L:
  BA:
    H: 0.556 [0.508, 0.602]
    L: 0.434 [0.384, 0.483]
  RB:
    L: 0.436 [0.387, 0.484]
    H: 0.553 [0.498, 0.602]

H-M:
  BA:
    H: 0.655 [0.641, 0.670]
    M: 0.591 [0.575, 0.607]
  RB:
    M: 0.609 [0.594, 0.622]
    H: 0.651 [0.636, 0.666]

M-L:
  BA:
    M: 0.406 [0.391, 0.422]
    L: 0.366 [0.351, 0.381]
  RB:
    L: 0.346 [0.331, 0.361]
    M: 0.398 [0.385, 0.412]

